In [7]:
import tensorflow as tf
import cv2

In [2]:
def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=False, name='residual_block', is_training=True):
    with tf.variable_scope(name):
        shortcut = x
        if conv_shortcut:
            shortcut = tf.layers.conv2d(x, filters, 1, strides=stride, padding='SAME', name='shortcut')
            shortcut = tf.layers.batch_normalization(shortcut, training=is_training, name='shortcut_bn')

        x = tf.layers.conv2d(x, filters, kernel_size, strides=stride, padding='SAME', name='conv1')
        x = tf.layers.batch_normalization(x, training=is_training, name='bn1')
        x = tf.nn.relu(x)

        x = tf.layers.conv2d(x, filters, kernel_size, padding='SAME', name='conv2')
        x = tf.layers.batch_normalization(x, training=is_training, name='bn2')
        x = x + shortcut
        x = tf.nn.relu(x)

        return x


In [9]:
def build_model(image1, image2, is_training=True):

    # Initial Convolution for both branches
    conv1_branch1 = tf.layers.conv2d(image1, 64, 3, strides=1, padding='SAME', name='conv1_branch1')
    conv1_branch2 = tf.layers.conv2d(image2, 64, 3, strides=1, padding='SAME', name='conv1_branch2')

    # Residual Block for both branches
    res_block_branch1 = residual_block(conv1_branch1, 64, name='residual_block_branch1', is_training=is_training)
    res_block_branch2 = residual_block(conv1_branch2, 64, name='residual_block_branch2', is_training=is_training)

    # Concatenate features from both branches
    concatenated_features = tf.concat([res_block_branch1, res_block_branch2], axis=-1)

    # Additional Residual Block on concatenated features
    res_block_concat = residual_block(concatenated_features, 128, name='residual_block_concat', is_training=is_training)

    # Further Convolution Layers
    conv2 = tf.layers.conv2d(res_block_concat, 128, 3, strides=1, padding='SAME', name='conv2')
    conv3 = tf.layers.conv2d(conv2, 128, 3, strides=1, padding='SAME', name='conv3')

    # Sigmoid Activation
    sigmoid_output = tf.nn.sigmoid(conv3)

    # Weighted Average with source images
    weighted_avg_output = (sigmoid_output + image1 + image2) / 3

    return weighted_avg_output

# Example usage
# Create 2 random 256x256 images


# Show image1 and image2
cv2.imshow('image1', image1)
cv2.imshow('image2', image2)
cv2.waitKey(0)
model_output = build_model(image1, image2, is_training=True)